# Simplified `Compressor`

In [ ]:
from pyturbo.systems import CompressorSimpleAero
from cosapp.drivers import NonLinearSolver

In [ ]:
cmp = CompressorSimpleAero('cmp')
solver = cmp.add_driver(NonLinearSolver('solver'))
solver.add_equation('fl_out.pt == 1.2e5').add_unknown('fl_in.W')

In [ ]:
cmp.shaft_in.power = 8e6
cmp.run_drivers()

print('mass flow : ', cmp.fl_in.W,' kg/s')
print('shaft power : ', cmp.shaft_in.power / 1e6, ' MW')
print('pt_ratio : ', cmp.pt_ratio)
print('Tt_ratio : ', cmp.Tt_ratio)